In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 50      # number of variables
num_ineq = 50     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msQuadratic
model = msQuadratic(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_exact_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:42:09<00:00, 61.30s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -15.884677               0.0     61.288965
std      0.339345               0.0      0.237985
min    -16.569278               0.0     61.031847
25%    -16.096610               0.0     61.167366
50%    -15.940147               0.0     61.273876
75%    -15.647801               0.0     61.351765
max    -15.008218               0.0     63.309573
Number of infeasible solution: 0


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("gurobi")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_rnd_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:38<00:00,  2.95it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean    -16.141183          1.883578      0.245323
std       0.318926          0.487851      0.069632
min     -16.979353          0.470149      0.156702
25%     -16.366533          1.517763      0.199059
50%     -16.138196          1.847186      0.231558
75%     -15.916087          2.209809      0.272832
max     -15.202193          3.384434      0.848458
Number of infeasible solution: 1000


In [16]:
df = pd.read_csv("result/cq_heur_rnd_50-50.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -16.172588          1.936195      0.332018
std      0.353489          0.453747      0.135587
min    -16.903971          0.896528      0.177150
25%    -16.391646          1.604608      0.230620
50%    -16.204917          1.904917      0.310094
75%    -15.898180          2.211433      0.404741
max    -15.383844          2.932958      0.848458
Number of infeasible solution: 100


## Heuristic - N1

In [9]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [10]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("gurobi")
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_heur_n1_50-50.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:14<00:00,  3.93it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.587427e+17               0.0      0.243883
std    1.495455e+17               0.0      0.111663
min   -1.628116e+01               0.0      0.157912
25%   -1.098950e+01               0.0      0.193062
50%    2.266936e+17               0.0      0.224638
75%    2.994127e+17               0.0      0.268512
max    5.661554e+17               0.0      2.894798
Number of infeasible solution: 0


In [17]:
df = pd.read_csv("result/cq_heur_n1_50-50.csv", index_col=0).iloc[:100]
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))

            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.675118e+17               0.0      0.280995
std    1.459043e+17               0.0      0.275393
min   -1.596815e+01               0.0      0.174124
25%   -1.028291e+01               0.0      0.210024
50%    2.448033e+17               0.0      0.239730
75%    2.971595e+17               0.0      0.270106
max    3.743183e+17               0.0      2.894798
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1485.66
Epoch 1, Validation Loss: -5.64
Epoch 2, Validation Loss: -9.07
Epoch 3, Validation Loss: -9.55
Epoch 4, Validation Loss: -9.64
Epoch 5, Validation Loss: -9.82
Epoch 6, Validation Loss: -9.87
Epoch 7, Validation Loss: -10.24
Epoch 8, Validation Loss: -9.79
Epoch 9, Validation Loss: -10.43
Epoch 10, Validation Loss: -9.97
Epoch 11, Validation Loss: -10.78
Epoch 12, Validation Loss: -10.11
Epoch 13, Validation Loss: -9.76
Epoch 14, Validation Loss: -10.43
Epoch 15, Validation Loss: -10.54
Epoch 16, Validation Loss: -11.11
Epoch 17, Validation Loss: -10.76
Epoch 18, Validation Loss: -11.18
Epoch 19, Validation Loss: -10.93
Epoch 20, Validation Loss: -10.53
Epoch 21, Validation Loss: -10.88
Epoch 22, Validation Loss: -11.63
Epoch 23, Validation Loss: -11.36
Epoch 24, Validation Loss: -11.32
Epoch 25, Validation Loss: -11.22
Epoch 26, Validation Loss: -11.46
Epoch 27, Validation Loss: -11.38
Epoch 28, Validation Loss: -11.84
Epoch 29, Validation Loss: -10.5

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lr_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 71.34it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -13.108997          0.000265      0.003088
std      0.249736          0.002647      0.001269
min    -13.655529          0.000000      0.001998
25%    -13.310327          0.000000      0.002234
50%    -13.130720          0.000000      0.002739
75%    -12.955102          0.000000      0.003194
max    -12.364275          0.026468      0.008058
Number of infeasible solution: 1


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1241.19
Epoch 1, Validation Loss: -7.48
Epoch 2, Validation Loss: -10.38
Epoch 3, Validation Loss: -11.51
Epoch 4, Validation Loss: -10.83
Epoch 5, Validation Loss: -11.64
Epoch 6, Validation Loss: -9.95
Epoch 7, Validation Loss: -11.33
Epoch 8, Validation Loss: -10.87
Epoch 9, Validation Loss: -11.41
Epoch 10, Validation Loss: -11.45
Epoch 11, Validation Loss: -11.53
Epoch 12, Validation Loss: -11.29
Epoch 13, Validation Loss: -11.65
Epoch 14, Validation Loss: -11.77
Epoch 15, Validation Loss: -10.87
Epoch 16, Validation Loss: -11.74
Epoch 17, Validation Loss: -11.14
Epoch 18, Validation Loss: -10.88
Epoch 19, Validation Loss: -11.39
Epoch 20, Validation Loss: -12.05
Epoch 21, Validation Loss: -11.94
Epoch 22, Validation Loss: -12.02
Epoch 23, Validation Loss: -11.78
Epoch 24, Validation Loss: -11.98
Epoch 25, Validation Loss: -11.52
Epoch 26, Validation Loss: -12.05
Epoch 27, Validation Loss: -11.74
Epoch 28, Validation Loss: -12.06
Epoch 29, Validation Loss

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_lt_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 71.38it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -12.905063          0.000497      0.003135
std      0.268591          0.004790      0.001240
min    -13.445088          0.000000      0.001001
25%    -13.093108          0.000000      0.002005
50%    -12.945923          0.000000      0.002999
75%    -12.749801          0.000000      0.003544
max    -12.279643          0.047884      0.008533
Number of infeasible solution: 2


### Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1248.58
Epoch 1, Validation Loss: -7.98
Epoch 2, Validation Loss: -11.57
Epoch 3, Validation Loss: -12.33
Epoch 4, Validation Loss: -12.14
Epoch 5, Validation Loss: -12.47
Epoch 6, Validation Loss: -12.71
Epoch 7, Validation Loss: -12.66
Epoch 8, Validation Loss: -12.58
Epoch 9, Validation Loss: -12.68
Epoch 10, Validation Loss: -12.67
Epoch 11, Validation Loss: -12.74
Epoch 12, Validation Loss: -12.57
Epoch 13, Validation Loss: -12.90
Epoch 14, Validation Loss: -12.74
Epoch 15, Validation Loss: -12.56
Epoch 16, Validation Loss: -12.80
Epoch 17, Validation Loss: -12.96
Epoch 18, Validation Loss: -12.35
Epoch 19, Validation Loss: -12.84
Epoch 20, Validation Loss: -12.71
Epoch 21, Validation Loss: -12.95
Epoch 22, Validation Loss: -12.91
Epoch 23, Validation Loss: -12.94
Epoch 24, Validation Loss: -13.25
Epoch 25, Validation Loss: -13.28
Epoch 26, Validation Loss: -13.11
Epoch 27, Validation Loss: -13.28
Epoch 28, Validation Loss: -13.32
Epoch 29, Validation Los

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_pr_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.16it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean   -14.155025          0.061082      0.001068
std      0.258000          0.104191      0.000516
min    -14.745079          0.000000      0.000000
25%    -14.324969          0.000000      0.001000
50%    -14.162546          0.000000      0.001001
75%    -14.008844          0.082220      0.001123
max    -13.439211          0.498455      0.002997
Number of infeasible solution: 44


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 128           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 1241.19
Epoch 1, Validation Loss: -6.30
Epoch 2, Validation Loss: -9.45
Epoch 3, Validation Loss: -10.25
Epoch 4, Validation Loss: -10.64
Epoch 5, Validation Loss: -10.44
Epoch 6, Validation Loss: -10.60
Epoch 7, Validation Loss: -10.44
Epoch 8, Validation Loss: -10.75
Epoch 9, Validation Loss: -10.84
Epoch 10, Validation Loss: -10.56
Epoch 11, Validation Loss: -10.85
Epoch 12, Validation Loss: -10.96
Epoch 13, Validation Loss: -11.07
Epoch 14, Validation Loss: -10.97
Epoch 15, Validation Loss: -10.66
Epoch 16, Validation Loss: -10.86
Epoch 17, Validation Loss: -10.96
Epoch 18, Validation Loss: -11.23
Epoch 19, Validation Loss: -10.81
Epoch 20, Validation Loss: -11.17
Epoch 21, Validation Loss: -11.17
Epoch 22, Validation Loss: -10.84
Epoch 23, Validation Loss: -11.09
Epoch 24, Validation Loss: -11.02
Epoch 25, Validation Loss: -10.84
Epoch 26, Validation Loss: -11.05
Epoch 27, Validation Loss: -11.29
Epoch 28, Validation Loss: -11.11
Epoch 29, Validation Loss

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/cq_st_50-50.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 92.11it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean   -11.982439               0.0      0.001486
std      0.303417               0.0      0.000654
min    -12.563549               0.0      0.000000
25%    -12.169167               0.0      0.001000
50%    -12.001996               0.0      0.001280
75%    -11.834228               0.0      0.002000
max    -11.253306               0.0      0.003054
Number of infeasible solution: 0
